In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [48]:
def scrape_letterboxd_reviews(movie_url, num_reviews):
    reviews = []
    page = 1
    while len(reviews) < num_reviews:        
        response = requests.get(f"{movie_url}/reviews/page/{page}/")
        soup = BeautifulSoup(response.content, 'html.parser')
        review_elements = soup.find_all('div', class_='film-detail-content')
        
        for element in review_elements:
            if len(reviews) >= num_reviews:
                break
            
            review_text_element = element.find('div', class_='body-text -prose collapsible-text')
            review_text = review_text_element.get_text(strip=True) if review_text_element else "No review text"
            
            rating_element = element.find('span', class_='rating')
            rating = rating_element.get_text(strip=True) if rating_element else "No rating"
            
            reviews.append({'review': review_text, 'rating': rating})
        
        page += 1

    return pd.DataFrame(reviews)

In [50]:
# Example usage
movie_url = 'https://letterboxd.com/film/deadpool-wolverine'  # Replace with actual movie URL
num_reviews = 10
reviews_df = scrape_letterboxd_reviews(movie_url, num_reviews)
#reviews_df.to_csv('data/reviews.csv', index=False)
reviews_df.head()

,review,rating
0,What the fucking hell.This was so amazing.I me...,★★★★
1,"If you didn’t like this movie, you’re not fun ...",★★★★★
2,low score bc of disney boot licking but a like...,★
3,"Wooooo, I’m boutta make a name for myself here",★★★★
4,R rated wolverine and it still feels weirdly r...,★★
